## Comprehensive Experiments

Run experiments on a larger data-set using basic ablation methods.

In [12]:
# Imports 
CWD = %pwd
CWD = str(CWD)

import json
from six.moves import cPickle
import os
import sys
import statistics

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline  


CWD

'/Users/Grendel/Desktop/ML/textmatters'

In [14]:
COCO_TEXT_PATH = os.path.join(CWD, 'coco-text')
sys.path.insert(0, COCO_TEXT_PATH)
import coco_text as ct
ct = ct.COCO_Text(COCO_PATH + 'COCO_Text.json')

loading annotations into memory...
0:00:02.701272
creating index...
index created!


In [3]:
"""
Load Coco data (Train)
"""
COCO_PATH = os.path.join(CWD,'data','coco')
COCO_ANNO_PATH = os.path.join(COCO_PATH, 'annotations')

# load the data from captions
with open(os.path.join(COCO_ANNO_PATH, 'captions_train2014.json')) as f:
    coco_captions = json.load(f)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_ids = []
img_captions = {}
for img_info in coco_captions['images']:
    mid = int(img_info['id'])
    img_ids.append(mid)
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = int(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
    img_captions[mid]['annotation'].append(cap_info)



In [35]:
"""
Split into three groups:
    1. Groups with no text instances
    2. Groups with relevant COCO-Text Instance 
       (the text in the instance also appears in the caption)
    3. Groups without relevant COCO-Text Instance
"""
no_texts     = []
rel_texts    = []
no_rel_texts = []
for mid in img_ids:    
    #  get text annotations
    anns = [ann for ann in ct.loadAnns(ct.getAnnIds(imgIds=mid)) if 'utf8_string' in ann ]
    if len(anns) == 0:
        no_texts.append(mid)
        continue
        
    # split text instances into words
    text_words = set([w.strip('.').upper() for ann in anns for w in ann['utf8_string'].split(' ')])

    # split captions into words
    caps = [ note['caption'] for note in img_captions[mid]['annotation']]
    caps_words = set([word.strip('.').upper() for word in (' '.join(caps).split())])

    if len(text_words & caps_words) > 0:
        rel_texts.append(mid)
    else:
        no_rel_texts.append(mid)

with open(os.path.join(CWD, 'input', 'no_texts_img_ids.pkl'),'w+') as f:
    cPickle.dump(no_texts, f)

with open(os.path.join(CWD, 'input', 'rel_texts_img_ids.pkl'),'w+') as f:
    cPickle.dump(rel_texts, f)
    
with open(os.path.join(CWD, 'input', 'no_rel_texts_img_ids.pkl'),'w+') as f:
    cPickle.dump(no_rel_texts, f)
    
print "No Texts:\t%d;\t\tRelevant Text:\t%d\t\tNo relevant text:\t%d"%(len(no_texts), len(rel_texts), len(no_rel_texts))

No Texts:	61247;		Relevant Text:	5556		No relevant text:	15980
